This code was taken from https://www.thepythoncode.com/article/extract-frames-from-videos-in-python. All credit goes to them.

I have no prior experience working with the OpenCV package, so I'm using their code to get a feel for it.

In [15]:
from datetime import timedelta
import cv2
import numpy as np
import os

In [16]:
def format_timedelta(td):
    '''Utility function to format timedelta objects in a cool way (e.g. 00:00:20.5)
    omitting microseconds and retaining milliseconds'''
    result = str(td)
    try:
        result, ms = result.split(".")
    except ValueError:
        return result + ".00".replace(":", "-")
    ms = int(ms)
    ms = round(ms/1e4)
    return f"{result}.{ms:02}".replace(":", "-")

def get_saving_frames_durations(cap, saving_fps):
    '''A function that returns the list of durations where to save the frames'''
    s = []
    # get the clip duration by dividing number of frames by the number of frames per second
    clip_duration = cap.get(cv2.CAP_PROP_FRAME_COUNT) / cap.get(cv2.CAP_PROP_FPS)
    # use np.arange() to make floating-point steps
    for i in np.arange(0, clip_duration, 1 / saving_fps):
        s.append(i)
    return s

In [30]:
def main(video_file):
    
    filename, _ = os.path.splitext(video_file)
    filename += "-opencv"
    
    # make a folder by the name of the video file
    if not os.path.isdir(filename):
        os.mkdir(filename)
    
    # read the video file
    cap = cv2.VideoCapture(video_file)
    
    # get the fps of the video
    fps = cap.get(cv2.CAP_PROP_FPS)
    
    # if the SAVIGN_FRAMES_PER_SECOND is above video fps, then set it to fps (as maximum)
    saving_frames_per_second = min(fps, SAVING_FRAMES_PER_SECOND)
    
    # get the list and duration spots to save
    saving_frames_durations = get_saving_frames_durations(cap, saving_frames_per_second)
    
    # start the loop
    count = 0
    while True:
        is_read, frame = cap.read()
        if not is_read:
            # break out of the loop if there are no frames to read
            break
        # get the duration by dividing the frame count by the FPS
        frame_duration = count / fps
        try:
            # get the earliest duration to save
            closest_duration = saving_frames_durations[0]
        except IndexError:
            # the list is empty, all duration frames were saved
            break
        if frame_duration >= closest_duration:
            # if closest duration is less than or equals the frame duration, 
            # then save the frame
            frame_duration_formatted = format_timedelta(timedelta(seconds=frame_duration))
            cv2.imwrite(os.path.join(filename, f"frame{frame_duration_formatted}.jpg"), frame) 
            # drop the duration spot from the list, since this duration spot is already saved
            try:
                saving_frames_durations.pop(0)
            except IndexError:
                pass
        # increment the frame count
        count += 1

In [31]:
SAVING_FRAMES_PER_SECOND = 10
main("football.mp4")